<a href="https://colab.research.google.com/github/nestorojeda/FashionMNIST/blob/master/FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import numpy as np
import tensorflow as tf
from keras import backend as K

from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD
from keras.datasets import fashion_mnist
from tensorflow.python.client import device_lib

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

num_classes = 10;

x_train = x_train.reshape(60000, 784) 
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255  # We normalize to have values between 0 and 1
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)  # onehot encoding
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


Iniciamos el procesamiento por GPU

In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15311710583304538963
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3470803646938812428
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4668976972355702485
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11140848026
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10829979642423250318
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


Comprobamos si Keras lo está utilizando


In [3]:
K.tensorflow_backend._get_available_gpus()


['/job:localhost/replica:0/task:0/device:GPU:0']

Creación de la red:

In [4]:
inputs = Input(shape=(784,))  
x = Dense(22, activation='sigmoid')(inputs)
predictions = Dense(10, activation='softmax')(x)

# This creates a model
model = Model(inputs=inputs, outputs=predictions)
sgd = SGD(lr=0.1)
model.compile(optimizer=sgd,
              loss='mse',  # Mean Squared Error
              metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 22)                17270     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                230       
Total params: 17,500
Trainable params: 17,500
Non-trainable params: 0
_________________________________________________________________
None


Entrenamiento con GPU

In [5]:
with tf.device('/gpu:0'):
  model.fit(x_train, y_train, epochs=20)  # starts training with 500 epochs

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
60000/60000 [==============================] - 6s 92us/step - loss: 0.0832 - acc: 0.4125
Epoch 2/20
60000/60000 [==============================] - 5s 90us/step - loss: 0.0669 - acc: 0.5959
Epoch 3/20
60000/60000 [==============================] - 5s 89us/step - loss: 0.0541 - acc: 0.6475
Epoch 4/20
60000/60000 [==============================] - 5s 90us/step - loss: 0.0466 - acc: 0.6965
Epoch 5/20
60000/60000 [==============================] - 5s 90us/step - loss: 0.0420 - acc: 0.7211
Epoch 6/20
60000/60000 [==============================] - 5s 89us/step - loss: 0.0389 - acc: 0.7367
Epoch 7/20
60000/60000 [==============================] - 5s 90us/step - loss: 0.0367 - acc: 0.7512
Epoch 8/20
60000/60000 [==============================] - 5s 89us/step - loss: 0.0349 - acc: 0.7653
Epoch 9/20
60000/60000 [==============================] - 5s 90us/step - loss: 0.0335 - acc: 0.7766
Epoch 10/20
60000/60000 [===========================

Resultado:

In [4]:
predictions = model.predict(x_test)
for p, l in zip(predictions, y_test):
    print(p,"->", l)

[0.00347309 0.00310756 0.00070016 0.00837855 0.00179194 0.1864897
 0.00210788 0.23547639 0.02463424 0.5338405 ] -> [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[1.2406925e-02 1.0539073e-03 8.5339278e-01 4.6026637e-03 3.8618311e-02
 4.9665314e-03 8.3371237e-02 5.0571177e-04 1.0673102e-03 1.4461641e-05] -> [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[9.3437862e-03 9.7329998e-01 1.5691487e-03 9.5849186e-03 2.9944116e-03
 7.6875131e-04 1.3867129e-03 5.8358768e-04 1.7572074e-04 2.9286050e-04] -> [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[6.7124437e-03 9.6629745e-01 1.9920669e-03 1.7233491e-02 3.6376452e-03
 1.3035035e-03 1.5448208e-03 6.7347835e-04 1.9955254e-04 4.0559177e-04] -> [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.11393725 0.00525565 0.1514938  0.03028644 0.02800552 0.00947242
 0.64418733 0.00299124 0.0133824  0.00098801] -> [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[3.89021486e-02 9.08563614e-01 6.64177537e-03 1.19180735e-02
 2.20543519e-02 1.33472332e-03 7.82352500e-03 1.43979676e-03
 7.84400210e-04 5.37578599e-04] -> [0. 1. 0. 0. 0